> Scatter Custom Symbol
# 第34课.自定义符号散点图
> Creating a custom ellipse symbol in scatter plot.

在散点图中自定义椭圆形符号。

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [3]:
%matplotlib widget
matplotlib.rcParams['font.family'] = ['DengXian', 'sans-serif']

[3]: 准备数据

In [4]:
rx, ry = 3., 1.
area = rx * ry * np.pi
theta = np.arange(0, 2 * np.pi + 0.01, 0.1)
verts = np.column_stack([rx / area * np.cos(theta), ry / area * np.sin(theta)])

In [5]:
x, y, s, c = np.random.rand(4, 30)
s *= 10**2.

In [6]:
fig, ax = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
ax.scatter(x, y, s, c, marker=verts)

## matplotlib.markers
> This module contains functions to handle markers. Used by both the marker functionality of plot and scatter.


> All possible markers are defined here:.

verts

> A list of (x, y) pairs used for Path vertices. The center of the marker is located at (0, 0) and the size is normalized, such that the created path is encapsulated inside the unit cell.



In [ ]:
#import matplotlib.markers as mmarkers

In [9]:
isinstance(verts, matplotlib.markers.MarkerStyle)

False

In [8]:
mko = matplotlib.markers.MarkerStyle(verts)

In [14]:
mko.get_path()

Path(array([[ 0.31830989,  0.        ],
       [ 0.31671966,  0.01059265],
       [ 0.31196488,  0.02107947],
       [ 0.30409305,  0.03135567],
       [ 0.29318282,  0.04131857],
       [ 0.27934321,  0.05086863],
       [ 0.26271249,  0.05991043],
       [ 0.24345683,  0.06835362],
       [ 0.22176863,  0.07611385],
       [ 0.1978646 ,  0.08311357],
       [ 0.17198357,  0.08928284],
       [ 0.14438413,  0.09456004],
       [ 0.11534206,  0.09889242],
       [ 0.08514752,  0.1022367 ],
       [ 0.05410222,  0.10455946],
       [ 0.02251635,  0.10583751],
       [-0.0092945 ,  0.10605805],
       [-0.04101248,  0.1052189 ],
       [-0.07232067,  0.10332844],
       [-0.10290627,  0.10040556],
       [-0.13246365,  0.09647945],
       [-0.16069751,  0.09158936],
       [-0.18732572,  0.08578413],
       [-0.21208224,  0.07912178],
       [-0.23471971,  0.07166887],
       [-0.25501193,  0.06349987],
       [-0.27275616,  0.05469639],
       [-0.2877751 ,  0.04534641],
       [-0.2999